In [512]:
from decimal import Decimal
from string import ascii_lowercase
import datetime

class TimeZone:
    def __init__(self, offset, name):
        self.name = name
        self.offset = offset
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise ValueError('Name must be a string.')
        self._name = value
        
    @property
    def offset(self):
        return self._offset
    
    @offset.setter
    def offset(self, value):
        if not isinstance(value, datetime.timedelta):
            raise TypeError('Offset must be a datetime.timedelta object.')
        self._offset = value
        
    def __repr__(self):
        return f'TimeZone(offset={self.offset}, name={self.name})'
        
    
        
    

class BankAccount:
    _monthly_interest_rate = None
    _tran_id = 0
    def __init__(self, accnt_no, f_name, l_name, tz_offset=None, balance=None):
        self.accnt_no = accnt_no
        self.f_name = f_name
        self.l_name = l_name
        self.tz_offset = tz_offset
        self._balance = Decimal(0.00)
        if balance:
            self.deposit(balance)
        
    @classmethod
    def get_interest_rate(cls):
        if cls._monthly_interest_rate is None:
            raise ValueError('Interest rate has not been set.')
        return cls._monthly_interest_rate
    
    @classmethod
    def set_interest_rate(cls, value):
        try:
            value = Decimal(value)
        except:
            raise ValueError('Interest rate must be a decimal value.')
        cls._monthly_interest_rate = value
        return f'New monthly interest rate = {round(cls._monthly_interest_rate * 100, 2)}%'
    
    @classmethod
    def _generate_confirmation(cls, account_no, tran_type):
        cls._tran_id += 1
        return (
            f"{tran_type}-{account_no}-"
            f"{datetime.datetime.utcnow().strftime('%Y%m%d%H%M%S')}-"
            f"{cls._tran_id}"
        )
    
    @property
    def accnt_no(self):
        return self._accnt_no
    
    @accnt_no.setter
    def accnt_no(self, value):
        try:
            value = int(value)
        except:
            raise ValueError('Account number must be an integer value.')
        if value <= 0:
            raise ValueError('Account number must be greater than zero.')
        self._accnt_no = value
        
    @property
    def f_name(self):
        return self._f_name
    
    @f_name.setter
    def f_name(self, value):
        # If every character in the provided value (assuming it's a string) is not a
        # standard letter, a space, or an apostrophe, raise an exception.
        if not all(map(lambda x: x.lower() in ascii_lowercase + " '", str(value))):
            raise ValueError('First name must contain only letters, spaces, or apostrophes.')
        self._f_name = value
        
    @property
    def l_name(self):
        return self._l_name
    
    @l_name.setter
    def l_name(self, value):
        # If every character in the provided value (assuming it's a string) is not a
        # standard letter, a space, or an apostrophe, raise an exception.
        if not all(map(lambda x: x.lower() in ascii_lowercase + " '", str(value))):
            raise ValueError('Last name must contain only letters, spaces, or apostrophes.')
        self._l_name = value
        
    @property
    def full_name(self):
        return f'{self.f_name} {self.l_name}'
    
    @property
    def tz_offset(self):
        return self._tz_offset
    
    @tz_offset.setter
    def tz_offset(self, value):
        if not (isinstance(value, TimeZone) or value is None):
            raise TypeError('tz_offset must be a TimeZone object.')
        self._tz_offset = value
        
    @property
    def balance(self):
        return self._balance
    
    def deposit(self, amount):
        try:
            amount = round(Decimal(amount), 2)
        except:
            conf_no = self._generate_confirmation(self.accnt_no, 'X')
            print('Deposit amount must be a decimal value.')
            print('Confirmation number:', conf_no)
            return
        if amount <= 0:
            conf_no = self._generate_confirmation(self.accnt_no, 'X')
            print('Deposit amount must be greater than zero.')
            print('Confirmation number:', conf_no)
            return
        self._balance += amount
        conf_no = self._generate_confirmation(self.accnt_no, 'D')
        print(
            "Deposit accepted.\n"
            f"New balance = {self.balance}\n"
            f"Confirmation number = {conf_no}"
        )
        
    def withdrawal(self, amount):
        try:
            amount = round(Decimal(amount), 2)
        except:
            conf_no = self._generate_confirmation(self.accnt_no, 'X')
            print('Withdrawal amount must be a decimal value.')
            print('Confirmation number:', conf_no)
            return
        if amount <= 0:
            conf_no = self._generate_confirmation(self.accnt_no, 'X')
            print('Withdrawal amount must be greater than zero.')
            print('Confirmation number:', conf_no)
            return
        if self._balance - amount < 0:
            conf_no = self._generate_confirmation(self.accnt_no, 'X')
            print('Insufficient funds.')
            print('Confirmation number:', conf_no)
            return
            
        self._balance -= amount
        conf_no = self._generate_confirmation(self.accnt_no, 'W')
        print(
            "Withdrawal accepted.\n"
            f"New balance = {self.balance}\n"
            f"Confirmation number = {conf_no}"
        )
    
    def interest_deposit(self):
        deposit_amount = self.balance * self.get_interest_rate()
        return self.deposit(deposit_amount)
    
    @staticmethod
    def parse_conf_no()
        

- each deposit and withdrawal must generate a **confirmation number** composed of:
    - the transaction type: `D` for deposit, and `W` for withdrawal, `I` for interest deposit, and `X` for declined (in which case the balance remains unaffected)
    - the account number
    - the time the transaction was made, using UTC
    - an incrementing number (that increments across all accounts and transactions)
    - for (extreme!) simplicity assume that the transaction id starts at zero (or whatever number you choose) whenever the program starts
    - the confirmation number should be returned from any of the transaction methods (deposit, withdraw, etc)

- create a **method** that, given a confirmation number, returns:
    - the account number, transaction code (D, W, etc), datetime (UTC format), date time (in whatever timezone is specified in te argument, but more human readable), the transaction ID
    - make it so it is a nicely structured object (so can use dotted notation to access these three attributes)
    - I purposefully made it so the desired timezone is passed as an argument. Can you figure out why? (hint: does this method require any information from any instance?)

We also want a method that given the confirmation number returns an object with attributes:
- `result.account_number` --> `140568`
- `result.transaction_code` --> `D`
- `result.transaction_id` --> `124`
- `result.time` --> `2019-03-15 07:59:00 (MST)`
- `result.time_utc` --> `2019-03-15T14:59:00`

In [513]:
delta = datetime.timedelta(hours=3, minutes=30)

In [514]:
tz = TimeZone(delta, 'MST')

In [515]:
tz.name

'MST'

In [516]:
ba = BankAccount(12345, 'Henry', 'Kissinger', tz, 5000)

Deposit accepted.
New balance = 5000.00
Confirmation number = D-12345-20210306000424-1


In [517]:
ba.tz_offset

TimeZone(offset=3:30:00, name=MST)

In [518]:
ba._generate_confirmation(12345, 'D')

'D-12345-20210306000424-2'

In [519]:
bb = BankAccount(54321, 'Oswald', 'Cobblepot')

In [520]:
bb._generate_confirmation(54321, 'W')

'W-54321-20210306000424-3'

In [521]:
ba.deposit(23)

Deposit accepted.
New balance = 5023.00
Confirmation number = D-12345-20210306000424-4


In [522]:
ba.deposit(300)

Deposit accepted.
New balance = 5323.00
Confirmation number = D-12345-20210306000425-5


In [523]:
ba.withdrawal('hello')

Withdrawal amount must be a decimal value.
Confirmation number: X-12345-20210306000425-6


In [524]:
ba.deposit(0.1)

Deposit accepted.
New balance = 5323.10
Confirmation number = D-12345-20210306000425-7


In [525]:
bb.deposit(-6)

Deposit amount must be greater than zero.
Confirmation number: X-54321-20210306000425-8


In [526]:
ba.interest_deposit()

ValueError: Interest rate has not been set.

In [528]:
BankAccount.set_interest_rate(0.004)

'New monthly interest rate = 0.40%'

In [529]:
ba.interest_deposit()

Deposit accepted.
New balance = 5344.39
Confirmation number = D-12345-20210306000509-9


In [530]:
bb.interest_deposit()

Deposit amount must be greater than zero.
Confirmation number: X-54321-20210306000525-10


In [247]:
ba.set_interest_rate(0.005)

'New monthly interest rate = 0.50%'

In [248]:
ba.balance

Decimal('5000.10')

In [254]:
ba.interest_deposit()

'Deposit accepted. New balance = 5126.36'

In [255]:
ba.balance

Decimal('5126.36')

In [18]:
ba = BankAccount('12345', ['h', 'e', 'l', 'l', 'o'], 'z')

ValueError: First name must contain only letters, spaces, or apostrophes.

In [148]:
ba.accnt_no

12345

In [149]:
ba.accnt_no = 'hello'

ValueError: Account number must be an integer value.

In [150]:
ba.accnt_no

12345

In [151]:
ba.__dict__

{'_accnt_no': 12345,
 '_f_name': 't',
 '_l_name': 'z',
 'tz_offset': 0,
 'balance': 0}

In [152]:
ba.l_name = 'Van der Lynden'

In [153]:
ba.f_name = "O'Shaughnessey"

In [154]:
ba.l_name

'Van der Lynden'

In [155]:
ba.f_name

"O'Shaughnessey"

In [156]:
ba.full_name

"O'Shaughnessey Van der Lynden"

In [157]:
all(map(lambda x: x.lower() in ascii_lowercase + " '", test_string))

True

In [19]:
import datetime

In [61]:
td = datetime.timedelta(hours=5, minutes=30)

In [62]:
type(td)

datetime.timedelta

In [32]:
td.seconds

19800

In [33]:
td.total_seconds()

19800.0

In [54]:
steve_time = datetime.timezone(td, 'Steve Time')

In [60]:
steve_time.

AttributeError: 'datetime.timezone' object has no attribute 'td'

In [41]:
test_time = datetime.datetime.utcnow()

In [42]:
test_time

datetime.datetime(2021, 3, 5, 20, 54, 20, 26361)

In [120]:
test_dec = Decimal(.234)

In [129]:
round(test_dec, 2)

Decimal('0.23')

In [265]:
test_time = datetime.datetime.utcnow()

In [276]:
test_time.strftime('%Y%m%d%H%M%S')

'20210305223547'

In [434]:
try:
    1 / 0
    print('hello')
except:
    raise ValueError('darn') from None
finally:
    print('test')

test


ValueError: darn

In [435]:
try:
    1 / 0
    print('hello')
except:
    raise ValueError('darn')
finally:
    print('test')

test


ValueError: darn